In [57]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


################################################################################
## Pandas Stuff
################################################################################
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


################################################################################
## Music Stuff
################################################################################
from musicUtils import *
from myMusicDBMap import myMusicDBMap


################################################################################
## Pandas Stuff
################################################################################
import sys
from collections import Counter
print("Python: {0}".format(sys.version))


################################################################################
## Prints
################################################################################
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-04-26 22:02:48.742682


# My Music DataBase

In [2]:
mdb = myMusicDBMap(debug=True)
#mdb.getFullDBData()()

Creating myMusicDBMap()
   Loading my music db map: /anaconda3/envs/py37/musicdb/myMusicMap.p
   DB keys: ['Discogs', 'AllMusic', 'MusicBrainz', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
Counter({'Discogs': 3918, 'AllMusic': 3812, 'MusicBrainz': 3674, 'LastFM': 2777, 'RockCorner': 466, 'DatPiff': 458, 'AceBootlegs': 175, 'CDandLP': 80, 'RateYourMusic': 30, 'MusicStack': 5, 'MetalStorm': 0})


In [23]:
from timeUtils import clock, elapsed
from myMusicDBMap import myMusicDBMap

class myMusicAnalysis:
    def __init__(self, mdb, debug=False):
        print("=================== Analysis of My Music ===================")
        self.debug = debug
        self.mdb   = mdb
        self.mdb.show()
        
        self.df = None
        self.results = {}

        
    def getArtistUnMatchedAlbums(self):
        retval         = {}


        ######################################################################
        #### Loop Over Prime Directories
        ######################################################################
        for primeDir in getPrimeDirectories():
            artistPrimeDirMap = getArtistPrimeDirMap(primeDir)
            print("UnMatched Albums: {0}".format(primeDir))
            #if primeDir.startswith("B"):
            #    break


            ######################################################################
            #### Loop Over Artist Name <-> Prime Map Items
            ######################################################################
            for artistName, artistPrimeDirs in artistPrimeDirMap.items():
                retval[artistName] = {"MyAlbums": [], "Todo": [], "Unknown": [], "Random": [], "Volumes": artistPrimeDirs}
                myAlbums           = {}
                for dirval in artistPrimeDirs:
                    myAlbums["MyAlbums"] = getMyMusicAlbums(dirval, returnNames=True)
                    myAlbums["Todo"]     = getMyTodoMusicAlbums(dirval)
                    myAlbums["Unknown"]  = getMyUnknownMusicAlbums(dirval)
                    myAlbums["Random"]   = getMyRandomMusic(dirval)

                    for key in myAlbums.keys():
                        retval[artistName][key] += myAlbums[key]

                for key in retval[artistName].keys():
                    retval[artistName][key] = len(retval[artistName][key])


        return retval



    def getArtistMatchedAlbums(self):
        albumTypes     = getAlbumTypes(keys=True)
        retval         = {}


        ######################################################################
        #### Loop Over Prime Directories
        ######################################################################
        for primeDir in getPrimeDirectories():
            artistPrimeDirMap = getArtistPrimeDirMap(primeDir) 
            print("Matched Albums: {0}".format(primeDir))
            #if primeDir.startswith("B"):
            #    break




            ######################################################################
            #### Loop Over Artist Name <-> Prime Map Items
            ######################################################################
            for artistName, artistPrimeDirs in artistPrimeDirMap.items():


                ######################################################################
                #### Get My Albums
                ######################################################################
                albumTypeResults = {}
                artistMatchDirs = [setDir(artistPrimeDir, 'Match', forceExist=False) for artistPrimeDir in artistPrimeDirs if isDir(artistPrimeDir, 'Match')]
                for albumType in albumTypes:
                    albumTypeResults[albumType] = []
                    for db in self.mdb.getDBs():
                        albumDBTypeNames = getAlbumTypes(db, albumType)
                        for albumDBTypeName in albumDBTypeNames:
                            for artistMatchDir in artistMatchDirs:
                                albumDBTypeDir = setDir(artistMatchDir, albumDBTypeName, forceExist=False)
                                if isDir(albumDBTypeDir):
                                    #print(albumDBTypeDir)
                                    albumTypeResults[albumType] += [getDirBasics(x)[-1] for x in findDirs(albumDBTypeDir)]
                                    #print(len(albumTypeResults),'\t',albumTypeResults)
                    albumTypeResults[albumType] = len(list(set(albumTypeResults[albumType])))
                retval[artistName] = albumTypeResults


        return retval
    
    
    def analyze(self):
        start, cmt = clock("Analyzing Artist Albums")

        artistUnMatchedAlbums = self.getArtistUnMatchedAlbums()
        artistMatchedAlbums   = self.getArtistMatchedAlbums()    

        self.results   = {}
        artistNames = set(list(artistUnMatchedAlbums.keys()) + list(artistMatchedAlbums.keys()))
        for artistName in sorted(artistNames):
            artistMatchedAlbumResults   = artistMatchedAlbums.get(artistName)
            artistUnMatchedAlbumResults = artistUnMatchedAlbums.get(artistName)
            #print(artistName,'\t',artistMatchedAlbumResults,'\t',artistUnMatchedAlbumResults)

            self.results[artistName]          = artistUnMatchedAlbumResults
            self.results[artistName]["Match"] = artistMatchedAlbumResults
            
        
        self.df = DataFrame(self.results).T            
            
        elapsed(start, cmt)
            
            
    def getDataFrame(self):
        if df is None:
            raise ValueError("DataFrame is not calculated!")
        return self.df
    
    
    def getResults(self):
        return self.results

In [77]:
## Basic stuff
%load_ext autoreload
%autoreload

from musicBase import myMusicBase
mmb = myMusicBase()
mmb.findArtistAlbums()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
My Music Base: ['/Volumes/Music/Matched', '/Volumes/Biggy/Matched']
Current Time is Sun Apr 26, 2020 22:20:06 for Finding All Artist Albums
Current Time is Sun Apr 26, 2020 22:20:11 for Done with Finding All Artist Albums
Process [Done with Finding All Artist Albums] took 5 seconds.


In [78]:
df = mmb.getDF()

In [80]:
for prime,pdf in df.groupby("Prime"):
    x = pdf
    break

In [82]:
x["Unmatched"]

A Flock Of Seagulls        1
A Global Threat            0
A Great Big World          1
A Tribe Called Quest       0
A*Teens                    0
A-Mafia                    2
A-Set                      0
A.D.O.R.                   0
ABBA                       0
ABC                        0
AC-DC                      1
AFI                        0
AIR                        1
AJR                        1
ATB                        1
Aaron Neville              1
Abdullah                   0
Abigor                     0
Abominator                 0
Above The Law              0
Abscess                    0
Absolution                 0
Absurdus                   0
Accept                     3
Ace Of Base                1
Acker Bilk                 1
Action League              0
Adam And The Ants          0
Adam Ant                   0
Adam Lambert               0
Addrisi Brothers           0
Adele                      0
Adryan Russ                1
Aerosmith                  0
Aetherian     

In [76]:
dirval = Series(df.index).apply(mmb.getPrimeDirectory)
dirval
#df["Prime"] = dirval

0          A
1          A
2          A
3          A
4          A
5          A
6          A
7          A
8          A
9          A
10         A
11         A
12         A
13         A
14         A
15         A
16         A
17         A
18         A
19         A
20         A
21         A
22         A
23         A
24         A
25         A
26         A
27         A
28         A
29         A
30         A
31         A
32         A
33         A
34         A
35         A
36         A
37         A
38         A
39         A
40         A
41         A
42         A
43         A
44         A
45         A
46         A
47         A
48         A
49         A
50         A
51         A
52         A
53         A
54         A
55         A
56         A
57         A
58         A
59         A
60         A
61         A
62         A
63         A
64         A
65         A
66         A
67         A
68         A
69         A
70         A
71         A
72         A
73         A
74         A
75         A
76         A

# Find Unmatched Data

In [63]:
df["Unmatched"]

A Flock Of Seagulls                 1
A Global Threat                     0
A Great Big World                   1
A Tribe Called Quest                0
A*Teens                             0
A-Mafia                             2
A-Set                               0
A.D.O.R.                            0
ABBA                                0
ABC                                 0
AC-DC                               1
AFI                                 0
AIR                                 1
AJR                                 1
ATB                                 1
Aaron Neville                       1
Abdullah                            0
Abigor                              0
Abominator                          0
Above The Law                       0
Abscess                             0
Absolution                          0
Absurdus                            0
Accept                              3
Ace Of Base                         1
Acker Bilk                          1
Action Leagu

In [65]:
from pandas import Series

0          A
1          A
2          A
3          A
4          A
5          A
6          A
7          A
8          A
9          A
10         A
11         A
12         A
13         A
14         A
15         A
16         A
17         A
18         A
19         A
20         A
21         A
22         A
23         A
24         A
25         A
26         A
27         A
28         A
29         A
30         A
31         A
32         A
33         A
34         A
35         A
36         A
37         A
38         A
39         A
40         A
41         A
42         A
43         A
44         A
45         A
46         A
47         A
48         A
49         A
50         A
51         A
52         A
53         A
54         A
55         A
56         A
57         A
58         A
59         A
60         A
61         A
62         A
63         A
64         A
65         A
66         A
67         A
68         A
69         A
70         A
71         A
72         A
73         A
74         A
75         A
76         A